In [246]:
import numpy as np
from numpy.linalg.linalg import norm, inv, cholesky, solve, qr, lstsq
from scipy.linalg import lu, eigvals
from sklearn.preprocessing import normalize
from time import process_time
from math import inf

n = 10; m = 8;
data = np.random.rand(n,m); data = np.maximum(data, np.zeros((n,m)))
k = np.int(np.floor(np.math.sqrt(np.max([n,m]))))
W = np.random.rand(n,k)
H = np.random.rand(k,m) 

maxiter = 200
tol = 1e-6
rho = 1e-3

W = np.maximum(W, rho)
H = np.maximum(H, rho)


array([[0.33175793, 0.39199865, 0.4373132 ],
       [0.18262435, 0.93461861, 0.70944532],
       [0.22283466, 0.64417073, 0.47550337],
       [0.79821745, 0.87565398, 0.68196597],
       [0.01370657, 0.9784358 , 0.06622164],
       [0.67507249, 0.47406148, 0.23010109],
       [0.72714257, 0.34930401, 0.01073209],
       [0.04354961, 0.43916842, 0.44082348],
       [0.78048189, 0.82354847, 0.3143967 ],
       [0.13272091, 0.2310466 , 0.91095381]])

In [2]:
def get_it(b, A):

    # 
    # This is a supplemental function for function nmf_alt_lsq_as.  It's purpose is to solve
    # Y = WH for W when given Y, H or for H when given Y, W.
    # 
    # Input:
    #     b = a numpy array size n-by-m, the RHS in AX = b
    #     A = a numpy array size n-by-k or k-by-m, the constant of the LHS of AX = b
    #
    # Output:
    #     X = a numpy array, either W or H
    #

    # SPD check: if eigenvalues are negative or imaginary, we initialize matrix differently
    AAT = A.T @ A
    if any(np.isreal(eigvals(AAT))) or any(eigvals(AAT) <= 0) :

        new_vals = np.random.randint(low = 1, high = 100, size = AAT.shape[0])
        q,_ = qr(AAT)
        AAT = q.T @ np.diag(new_vals) @ q
        AAT = np.maximum(AAT, rho)

    r = b.shape[1]
    R = cholesky(AAT)
    D = solve(R.T, (A.T @ b))
    X = np.zeros((k, r)); 
    # initialize x to be random, nonnegative vector
    temp = np.random.rand(k, 1)
    seq = [i for i in range(r)]

    # finds the NMF for d = R*x, one column of X at a time
    temp = lsqnoneg(R, D[:, seq[0]], temp)
    X[:, seq[0]] = temp.reshape((3,))
    for i in range(1, r):
        temp = lsqnoneg(R, D[:, seq[i]], temp)
        X[:, seq[i]] = temp.reshape((3,))
    
    # return W or H 
    return X

In [3]:
def lsqnoneg(C, d, x):

    # 
    # This is a supplemental function for function nmf_alt_lsq_as.  It's purpose is to solve the
    # convex problem min_x{||d - Cx||_f^2} using the active set strategy described in 
    #
    # Gu, R., Du, Q., & Billinge, S. J. (2021). A fast two-stage algorithm for non-negative matrix 
    # factorization in streaming data. arXiv preprint arXiv:2101.08431.
    #
    # is used here.
    # 
    # Input:
    #     C = a numpy array found from a Cholesky factorization, a constant
    #     d = a numpy array, also a constant
    #     x = a numpy array, to be determined in the run of this function
    #     nZeroes = a dummy variable
    #
    # Output:
    #     z = a numpy array, determined in the run of this function
    #

    d = d.reshape((k,1))

    Z = x <= tol; indx_Z = Z.tolist(); Z = [elem[0] for elem in indx_Z]
    P = x > tol; indx_P = P.tolist(); P = [elem[0] for elem in indx_P]
    wz = np.zeros((k, 1)); 
    z = np.zeros((k, 1)); g, _, _, _ = lstsq(C[:,P], d, rcond = None); z[P] = g;
        
    for outiter in range(20):
        for inneriter in range(10):
            
            indx = z <= 0; indx = [elem[0] for elem in indx]; Q = indx and P

            if x[Q].size == 0:
                alpha = rho
            else:
                alpha = np.min(x[Q]/(x[Q] - z[Q] + rho))
                
            x = x + alpha*(z - x)
            indx = np.abs(x) < tol; indx = [elem[0] for elem in indx]; Z = (indx and P) or Z
            P = x > tol; indx = P.tolist(); P = [elem[0] for elem in indx]
            z = np.zeros((k, 1)); g, _, _, _ = lstsq(C[:,P], d, rcond = None); z[P] = g

        x = np.array(z); w = C.T @ (d - C @ x)
        indx = w[Z] > tol; indx = [elem[0] for elem in indx]

        z = np.zeros((k, 1)); 
        wz[P] = -inf; 
        if (np.sum(Z) >  0):
            wz[Z] = w[Z]
        
        t = np.argmax(wz); P[t] = True; Z[t] = False;
        g, _, _, _ = lstsq(C[:,P], d, rcond = None); z[P] = g
            

    return z

# Phase 1

Stage 1 is the least squares step with active set method.

In [4]:
def nmf_alt_lsq_as(Y, W, H):

    # grad_W = ((W @ H) - Y) @ H.T
    # grad_H = W.T @ ((W @ H) - Y)

    flag = 0
    # iter = 0

    for iter in range(maxiter):

        W = get_it(Y.T, H.T); W = W.T
        H = get_it(Y, W)

        # normalize
        W = normalize(W, axis = 0)
        H = normalize(H, axis = 1)

        #update gradients
        # grad_W = ((W @ H) - Y) @ H.T
        # grad_H = W.T @ ((W @ H) - Y)

        # checking for ??
        # sigma ??
        if ( (iter > 1 and np.sum(W > 0) == n*k) and (np.sum(H > 0) == k*m) or iter == 50 ):

            flag = flag + 1

            if flag == 2:
                break;
        else:

            flag = 0

    return [W, H]

In [5]:
# when run after the other functions are compiled, this method took 1m 7.3s.
[W, H] = nmf_alt_lsq_as(data, W, H)

# Phase 2
Stage 2 is the interior precorrector method.

In [247]:
def nmf_int_precor(data, W, H):

    # initiate eta
    eta = 0

    # initiate r and s
    grad_W = ((W @ H) - data) @ H.T
    grad_H = W.T @ ((W @ H) - data)
    r = np.ones((k*n, 1))*np.max(np.abs(grad_W))
    s = np.ones((k*m, 1))*np.max(np.abs(grad_H))

    # initiate mu and sigma
    mu = inf
    sigma = inf

    for i in range(maxiter):

        # set up for tolerance check of E(w_k, h_k, r_k, s_k) in algorithm 2
        grad_W = ((W @ H) - data) @ H.T; gra_W = grad_W.reshape((k*n, 1));
        grad_H = W.T @ ((W @ H) - data); gra_H = grad_H.reshape((k*m, 1))
        w = W.reshape((k*n, 1)); h = H.reshape((k*m, 1))

        temp1 = np.vstack((gra_W - r,gra_H - s))
        temp2 = np.vstack((w * r, h * s))

        # tolerance check
        if np.max([norm(temp1), norm(temp2)]) < tol:
            break

        # initiate mu, eta
        mu = 0
        nu = 0
        eta = sigma < 1e-2

        # Finding Q2 in equation (5)
        if i == 0:
            Q2 = np.zeros((k*m, k*m))

        for j in range(m):
            
            indx = np.r_[j*k:(j*k+k)]
            values = s[indx]/(h[indx] + rho); D = np.diagflat(values)
            Q2[j*k:(j*k+k), j*k:(j*k+k)] = (W.T @ W) + D

        # Finding CP in equation (?)
        if i == 0:
            CP1i = np.zeros((m*k, n*k)); Ri = np.zeros((n*k, k)); Rit = Ri;

        Rx = r/(w + rho)

        for j in range(n):
            
            indx = np.r_[j*k:(j*k+k)]
            values = Rx[indx] + rho; D = np.diagflat(values)
            Q1 = inv(cholesky( (H @ H.T) + D ))

            Ri[indx,:] = Q1; Rit[indx,:] = Q1.T
            temp1 = W[j,:].reshape((1,k)); 
            temp2 = ((Q1.T.reshape((k*k,1))) @ (eta*(W @ H - data)[j,:].reshape((1,m)))).reshape((k,m*k))
            F = np.tile(temp1.T, (m, k)) * np.tile((Q1.T @ H), (k, 1)).T.reshape((k, k*m)).T + temp2.T
            CP1i[:, indx] = F


        # prediction step
        
        beq1 = -gra_W
        beq2 = -gra_H
        reshape_beq1 = (np.tile(beq1.reshape((k, n)), (k, 1))).reshape((k, k*n)).T
        P1ib1 = np.sum(Ri * (reshape_beq1), axis = 1)
        [P, L, U] = lu(Q2 - (CP1i @ CP1i.T))
        CP = (CP1i @ P1ib1).reshape((k*m, 1))
        g, _, _, _ = lstsq(beq2 - CP, L, rcond = None)
        solu2, _, _, _ = lstsq(g.T, U, rcond = None); solu2 = solu2.T
        reshape_diff = np.tile(P1ib1.reshape((n*k, 1)) - (CP1i.T @ solu2), (1, k))
        solu1 = np.sum((Ri * reshape_diff), axis = 1); solu1 = solu1.reshape((n*k, 1))
        dr = ((mu/w) - r) - (r * solu1/w)
        ds = ((nu/h) - s) - (s * solu2/h)

        solutions = -np.vstack([solu1, solu2])/np.vstack([w, h])
        derivs = -np.vstack([dr, ds])/np.vstack([r, s])
        step1 = np.min([1/np.max(solutions), 1])
        step2 = np.min([1/np.max(derivs), 1])
        mu_aff = ( (np.vstack([w, h]) + step1*np.vstack([w,h])).T @ (np.vstack([r,s]) + step2*np.vstack([dr,ds])))/(n*k + m*k)
        mu = (np.vstack([w,h]).T @ np.vstack([r,s]))/(n*k + m*k)
        sigma = np.min([((mu_aff/mu)**3), 0.99])

        aff_1 = solu1 * dr; aff_2 = solu2 * ds;

        beq1 = (mu*sigma - aff_1)/(w - gra_W);
        beq2 = (mu*sigma - aff_2)/(h - gra_H);

        reshape_beq1 = (np.tile(beq1.reshape((k, n)), (k, 1))).reshape((k, k*n)).T
        P1ib1 = np.sum(Ri * (reshape_beq1), axis = 1)
        [P, L, U] = lu(Q2 - (CP1i @ CP1i.T))
        CP = (CP1i @ P1ib1).reshape((k*m, 1))
        g, _, _, _ = lstsq(beq2 - CP, L, rcond = None)
        solu2, _, _, _ = lstsq(g.T, U, rcond = None); solu2 = solu2.T
        reshape_diff = np.tile(P1ib1.reshape((n*k, 1)) - (CP1i.T @ solu2), (1, k))
        solu1 = np.sum((Ri * reshape_diff), axis = 1); solu1 = solu1.reshape((n*k, 1))
        dr = ((mu/w) - r) - (r * solu1/w)
        ds = ((nu/h) - s) - (s * solu2/h)

        # update step-sizes
        solutions = -np.vstack([solu1, solu2])/np.vstack([w, h])
        derivs = -np.vstack([dr, ds])/np.vstack([r, s])
        step1 = np.min([0.9/np.max(solutions), 1])
        step2 = np.min([0.9/np.max(derivs), 1])

        # update the gradients for W and H
        dW = solu1.reshape((k,n)).T
        dH = solu2.reshape((k,m))

        # update W, H, r, and s using gradient update
        W1 = W + dW*step1
        H1 = H + dH*step1
        r1 = r + dr*step2
        s1 = s + ds*step2

        # calculate phi(W, H)
        logW = np.log(W); logH = np.log(H)
        phi = (0.5)*(norm(W @ H - data, ord = 'fro')) - mu*sigma*np.sum(logW, axis = None) - mu*sigma*np.sum(logH, axis = None)

        # calculate grad_phi(W,H)
        grad_phi = np.vstack([1/(gra_W - mu*sigma*w), 1/(gra_H - mu*sigma*h)])

        d0 = np.vstack([dW.reshape((k*n, 1)), dH.reshape((k*m, 1))])
        if ((grad_phi.T @ d0) >= 0):
            continue

        W = np.maximum(W1, rho)
        H = np.maximum(H1, rho)
        r = r1 
        s = s1

    return [W, H]

In [249]:
[W, H] = nmf_int_precor(data, W, H)

In [253]:
norm(W @ H - data, ord = 'fro')/norm(data, ord = 'fro')

0.6980688653616256